# Diagnosis System version 2 -- test
This is a testing version for our diagnosis system. Virtual patients are generated and diagnosed with the system, and the accuracy is calculated.

In [1]:
# load data
import pandas as pd
import numpy as np

WM = pd.read_csv('../WeightMatrix/Dis_Sym_Matrix.csv', index_col=0)

dis2sym = pd.read_csv('../WeightMatrix/dis_symptom.csv', header=None)

dis2sym.fillna(method='ffill',inplace=True)

umls_dis = {}
umls_sym = {}
dis_num = {}
for i in dis2sym.index:
    temp = dis2sym.loc[i][0]
    items = temp.split('^')
    item = items[0].strip('UMLS:').split('_')
    if len(item) != 2: continue
    umls_dis[item[0]] = item[1]
    dis_num[item[0]] = int(dis2sym.loc[i][1])
for i in dis2sym.index:
    temp = dis2sym.loc[i][2]
    items = temp.split('^')
    item = items[0].strip('UMLS:').split('_')
    if len(item) != 2: continue
    umls_sym[item[0]] = item[1]
    
rev_sym = {v: k for k, v in umls_sym.items()}
rev_dis = {v: k for k, v in umls_dis.items()}



In [2]:
patients = {}

for i in WM.index:
    patients[i] = []
    for j in WM.columns:
        if WM.loc[i][j] != 0:
            patients[i].append(j)
#patients

In [59]:
#patients

In [88]:
def SelectedMatrix(sym):
    selected = WM[WM[sym] != 0]
    selected = selected.drop(columns=[sym])
    for c in selected.columns:
        if sum(selected[c]) == 0:
            selected.drop(columns=[c],inplace=True)
    return selected
    

In [67]:
def renorm(dia):
    dia.sort_values(ascending=False, inplace=True)
    temp = dia
    
    s = sum(temp[:5])
    return temp/s

In [102]:
def diagnosis(dis, sym):
        
    selected = SelectedMatrix(sym)
    
    #The response vector
    res = pd.Series(index=WM.columns, data=[0]*len(WM.columns))
    res[sym] = 1
    
    #Diagnosis process
    while True:
        dia = WM.dot(res)
        dia.sort_values(ascending=False, inplace=True)
        #print(sorted(dia,reverse=True))
        if max(dia) > 0.3: 
            #dia = renorm(dia)
            if dia.keys()[0] == dis:
                return 1
            else:
                return -1
        if len(selected) == 1 or len(selected.columns) == 1:
            return 0
            
        #choose the most relevant symptom to ask: The symptom that are least shared with other diseases
        next_i = selected.columns[0]
        s = 0
        sdia = dia[selected.index]
        sdia.sort_values(ascending=False, inplace=True)
        d0 = sdia.keys()[0]
        d1 = sdia.keys()[1]
        for i in selected.columns:            
            if selected.loc[d0][i] > 0:
                next_i = i
                '''if 0 in selected[i].value_counts():
                    pri = selected[i].value_counts()[0]
                    if pri > s:
                        s = pri
                        next_i = i'''
         
        selected.drop(columns=[next_i], inplace = True)
        if next_i in patients[dis]:
            res[next_i] = 1
        else:
            res[next_i] = 0
            selected.drop(selected.index[0], inplace=True)
    

In [103]:
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

total: 1854
Correct: 490
Wrong: 473
fail: 891


In [52]:
#cube, 0.8
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

total: 1854
Correct: 1086
Wrong: 43
fail: 725


In [53]:
#cube, 0.5
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

total: 1854
Correct: 1086
Wrong: 43
fail: 725


In [56]:
#square, 0.5
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

total: 1854
Correct: 1330
Wrong: 238
fail: 286


In [65]:
#quad, 0.8
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

total: 1854
Correct: 1327
Wrong: 153
fail: 374


In [ ]:
1327/1854

In [92]:
import pandas as pd
a = pd.Series(index = [1,2,3,4], data = [1,2,3,4])
b = pd.DataFrame(index = [1,2,3], data = [2,3,4])

In [95]:
a[b.index]

1    1
2    2
3    3
dtype: int64